In [2]:
!pip install sqlalchemy psycopg2 pandas
import pandas as pd
from sqlalchemy import create_engine

# Define the connection string (replace placeholders with actual values)
database_type = 'postgresql'
username = 'your_username'
password = 'your_password'
host = 'your_host'
port = 3306  
database = 'sakila'

# Create the engine
engine = create_engine(f'{database_type}://{username}:{password}@{host}:{port}/{database}')

# Test the connection
try:
    with engine.connect() as connection:
        print("Connection established successfully!")
except Exception as e:
    print(f"Error: {e}")

def rentals_month(engine, month, year):
    query = f"""
    SELECT rental_id, rental_date, inventory_id, customer_id, return_date, staff_id
    FROM rental
    WHERE EXTRACT(MONTH FROM rental_date) = {month} AND EXTRACT(YEAR FROM rental_date) = {year}
    """
    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
    return df

def rental_count_month(df, month, year):
    rental_count = df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02d}_{year}')
    return rental_count

def compare_rentals(df1, df2, month1, year1, month2, year2):
    merged_df = pd.merge(df1, df2, on='customer_id', how='outer', suffixes=(f'_{month1:02d}_{year1}', f'_{month2:02d}_{year2}'))
    merged_df.fillna(0, inplace=True)
    merged_df['difference'] = merged_df[f'rentals_{month2:02d}_{year2}'] - merged_df[f'rentals_{month1:02d}_{year1}']
    return merged_df

# Retrieve data for May and June 2005
df_may = rentals_month(engine, 5, 2005)
rental_count_may = rental_count_month(df_may, 5, 2005)

df_june = rentals_month(engine, 6, 2005)
rental_count_june = rental_count_month(df_june, 6, 2005)

# Compare rentals between May and June 2005
comparison_df = compare_rentals(rental_count_may, rental_count_june, 5, 2005, 6, 2005)
print(comparison_df.head())


Error: (psycopg2.OperationalError) could not translate host name "your_host" to address: No such host is known. 

(Background on this error at: https://sqlalche.me/e/20/e3q8)
